# Task 1

In [33]:
import random
import pandas as pd
import numpy as np
SEED_VALUE = 13
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
n=24
attendance_raw = [
    {
        "student_id": f"S{i:03d}",
        "cohort": np.random.choice(["alpha", "beta", "gamma"]),
        "attended_sessions": np.random.randint(0, 7),
        "expected_sessions": 6
    }
    for i in range(1, n + 1)
]

In [34]:
attendace = pd.DataFrame(attendance_raw)

In [35]:
"""import random
import pandas as pd
import numpy as np
n=24
attendance_raw ={
        "student_id": [f"S{i:03d}" for i in range(1, n + 1)],
    "cohort": np.random.choice(["alpha", "beta", "gamma"], size=n),
    "attended_sessions": np.random.randint(0, 7, size=n),
    "expected_sessions": [6] * n
    }
"""  
#alternative version 

'import random\nimport pandas as pd\nimport numpy as np\nn=24\nattendance_raw ={\n        "student_id": [f"S{i:03d}" for i in range(1, n + 1)],\n    "cohort": np.random.choice(["alpha", "beta", "gamma"], size=n),\n    "attended_sessions": np.random.randint(0, 7, size=n),\n    "expected_sessions": [6] * n\n    }\n'

In [36]:
print(attendace.head(5))
attendace.info()

  student_id cohort  attended_sessions  expected_sessions
0       S001  gamma                  0                  6
1       S002  gamma                  0                  6
2       S003  gamma                  2                  6
3       S004  alpha                  1                  6
4       S005  alpha                  2                  6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   student_id         24 non-null     object
 1   cohort             24 non-null     object
 2   attended_sessions  24 non-null     int64 
 3   expected_sessions  24 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 900.0+ bytes


# Task 2

In [56]:
attendance_indexed= attendace.set_index("student_id")
attendance_indexed.head()

,cohort,attended_sessions,expected_sessions
student_id,,,
S001,gamma,0,6
S002,gamma,0,6
S003,gamma,2,6
S004,alpha,1,6
S005,alpha,2,6


In [57]:
excused_data = {
    'S001': 1,
    'S003': 2, 
    'S010': 1,
    'S015': 1, 
    'S020': 2, 
    'S022': 1,  
    'S025': 3,  
    'S046': 1,  
    'S099': 2,  
    'S100': 1   
}
excused_absences = pd.Series(excused_data)

In [58]:
attendance_indexed["adjusted_attendance"] = attendance_indexed["attended_sessions"] + excused_absences

In [59]:
attendance_indexed.head()

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,gamma,0,6,1.0
S002,gamma,0,6,NaN
S003,gamma,2,6,4.0
S004,alpha,1,6,NaN
S005,alpha,2,6,NaN


In [60]:
attendance_indexed["adjusted_attendance"]=attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"] )

In [61]:
attendance_indexed.head()

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,gamma,0,6,1.0
S002,gamma,0,6,0.0
S003,gamma,2,6,4.0
S004,alpha,1,6,1.0
S005,alpha,2,6,2.0


In [62]:
attendance_indexed[["attended_sessions", "adjusted_attendance"]].head(5)
#for checking

,attended_sessions,adjusted_attendance
student_id,,
S001,0,1.0
S002,0,0.0
S003,2,4.0
S004,1,1.0
S005,2,2.0


# Task 3

In [69]:
attendance_indexed.iloc[0, 0] = "  Alpha " 
attendance_indexed.iloc[1, 0] = "BETA"     
attendance_indexed.iloc[2, 0] = " gamma  "

In [70]:
attendance_indexed['cohort'].unique()
#before cleaning

array(['  Alpha ', 'BETA', ' gamma  ', 'alpha', 'gamma', 'beta'],
      dtype=object)

In [72]:
attendance_indexed['cohort'] = attendance_indexed['cohort'].str.strip().str.lower()

In [73]:
attendance_indexed['cohort'].unique()
#after cleaning

array(['alpha', 'beta', 'gamma'], dtype=object)

# Task 4

In [88]:
low_attendance=attendance_indexed[(attendance_indexed['attended_sessions']<attendance_indexed['expected_sessions'])]
low_attendance.head()

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,alpha,0,6,1.0
S002,beta,0,6,0.0
S003,gamma,2,6,4.0
S004,alpha,1,6,1.0
S005,alpha,2,6,2.0


In [89]:
av_low_attendance = low_attendance.groupby('cohort')['attended_sessions'].mean()
av_low_attendance

cohort
alpha    2.125000
beta     2.000000
gamma    2.571429
Name: attended_sessions, dtype: float64

In [90]:
expected_cohorts = ['alpha', 'beta', 'gamma']
current_cohorts = av_low_attendance.index.tolist()
if all(c in expected_cohorts for c in current_cohorts):
    print("\n✅ Confirmed: Cohort names are clean and inconsistencies have been resolved.")
else:
    print("\n⚠️ Warning: There are still uncleaned or unexpected values in the cohort names.")


✅ Confirmed: Cohort names are clean and inconsistencies have been resolved.


# Task 5

In [91]:
attendance_indexed["attendance_ok"] = attendance_indexed["attended_sessions"]>=attendance_indexed['expected_sessions']

In [95]:
attendance_indexed.head(8)

,cohort,attended_sessions,expected_sessions,adjusted_attendance,attendance_ok
student_id,,,,,
S001,alpha,0,6,1.0,False
S002,beta,0,6,0.0,False
S003,gamma,2,6,4.0,False
S004,alpha,1,6,1.0,False
S005,alpha,2,6,2.0,False
S006,gamma,4,6,4.0,False
S007,gamma,2,6,2.0,False
S008,gamma,6,6,6.0,True


In [96]:
validation_check = attendance_indexed.loc[low_attendance.index, 'attendance_ok']
if not validation_check.any():
    print("\n✅ Validation Successful: Every row in low_attendance has attendance_ok as False.")
else:
    print("\n⚠️ Validation Failed: Some rows in low_attendance have attendance_ok as True.")


✅ Validation Successful: Every row in low_attendance has attendance_ok as False.
